Beast Cookbook: Use the LLM to generate a full, fictional "Mr. Beast Cookbook" with recipes that incorporate elements from his challenges and stunts. The recipes could have humorous, pun-filled names and absurd, over-the-top ingredients lists.

The "Beast Cookbook" idea is a creative way to engage with Mr. Beast's content by generating a fictional cookbook inspired by his over-the-top challenges and stunts. By leveraging a large language model (LLM), you can create a collection of wild, whimsical recipes that capture the spirit of his videos.

The LLM can help generate humorous recipe names, absurd ingredients lists, and entertaining instructions that reference specific moments from Mr. Beast's channel. The cookbook could be organized into sections based on different types of challenges or themes, and include engaging elements like illustrations, pun-filled captions, or fictitious testimonials.

The "Beast Cookbook" could serve as a standalone piece of content or a promotional tool to drive engagement with Mr. Beast's channel. It's a clever way to transform video content into a new, amusing format that showcases the creativity and humor of his brand, and can be enjoyed by his fans in various forms, such as a digital download or limited-edition physical merchandise.

In [3]:
%pip install --upgrade --quiet  sentence_transformers > /dev/null

Note: you may need to restart the kernel to use updated packages.


In [4]:
! pip install -U llama-cpp-python langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.9/36.9 MB 62.4 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install gradio -qq

In [28]:
!pip install langchain_mistralai -qq

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [30]:
import os
import numpy as np
import pandas as pd
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from typing import List

In [17]:
df = pd.read_csv('mrbeast.csv')
df.head()


,title,transcript,views,length,rating,description,keywords,publish_date,thumbnail_url,channel_title,channel_id
0,THE WORST SONG ON YOUTUBE *CRINGE*,THE WORST SONG ON YOUTUBE *CRINGE*\n\n What's ...,771458,247,NaN,This is a song about bad teammates in COD \n\n...,"['the worst song on youtube', 'cringiest song ...",2016-08-08,https://i.ytimg.com/vi/yozl6enFC-w/sddefault.jpg,MrBeast,UCX6OQ3DkcsbYNE6H8uQQuVA
1,Surviving 24 Hours Straight In The Bermuda Tri...,Surviving 24 Hours Straight In The Bermuda Tri...,152738695,701,NaN,THIS IS ONE OF THE CRAZIEST THINGS I'VE EVER D...,[],2019-11-09,https://i.ytimg.com/vi/D9lVNzyhYnc/sddefault.jpg,MrBeast,UCX6OQ3DkcsbYNE6H8uQQuVA
2,I Hope This Magic Trick Works,I Hope This Magic Trick Works\n\n Welcome back...,41925793,982,NaN,WE ATTEMPTED POPULAR MAGIC TRICKS AND THIS HAP...,"['magic tricks', 'magic trick', 'experiment', ...",2018-09-11,https://i.ytimg.com/vi/tBEBc4KQVsU/sddefault.jpg,MrBeast,UCX6OQ3DkcsbYNE6H8uQQuVA
3,Important Youtube Announcement (please watch),Important Youtube Announcement (please watch)\...,34218111,155,NaN,I CAN'T BELIEVE WE REACHED OUR GOAL!!!!! YOU G...,[],2019-12-28,https://i.ytimg.com/vi/0hVZOJCYBBM/sddefault.jpg,MrBeast,UCX6OQ3DkcsbYNE6H8uQQuVA
4,Every Challenge I've Ever Done,Every Challenge I've Ever Done\n\n Welcome to ...,820648,85686,NaN,This is basically my life in a nutshell\n\nSUB...,"['counting to 100000', 'reading the bee movie ...",2017-06-23,https://i.ytimg.com/vi/KcsRxHwGz7A/sddefault.jpg,MrBeast,UCX6OQ3DkcsbYNE6H8uQQuVA


In [18]:
print(df['transcript'].iloc[10][:100])


I Cleaned The World’s Dirtiest Beach #TeamSeas

 This is one of the dirtiest beaches in the entire w


In [19]:
local_llm = "mistral"


In [20]:
from langchain.docstore.document import Document

doc =  Document(page_content=df['transcript'].iloc[100], metadata={"source": "local"})

In [21]:
docs = []
for d in df['transcript']:
    doc =  Document(page_content=d, metadata={"source": "local"})
    docs.append(doc)

In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import LlamaCppEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings



text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=100
)
all_splits = text_splitter.split_documents(docs)

In [23]:
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Index
vectorstore = Chroma.from_documents(
    documents=all_splits,
    collection_name="rag-chroma",
    embedding=embedding,
)
retriever = vectorstore.as_retriever()

In [24]:
def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = retriever.get_relevant_documents(question)
    return {"keys": {"documents": documents, "question": question}}


In [25]:
import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_mistralai.chat_models import ChatMistralAI

local = False
def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]

    # Prompt
    prompt = hub.pull("rlm/rag-prompt")

    # LLM
    if local:
            llm = Ollama(model=local_llm, temperature=0.6)
    else:
        llm = ChatMistralAI(mistral_api_key='', model = 'mistral-tiny')

    # Post-processing
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    # Chain
    rag_chain = prompt | llm | StrOutputParser()

    # Run
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {
        "keys": {"documents": documents, "question": question, "generation": generation}
    }

In [26]:
def get_food_elements():
    state = {
        "keys":{
            "question": "Identify food items, ingredients, or eating challenges mentioned"
        }
    }
    state = retrieve(state)
    state = generate(state)
    food_elements = state['keys']['generation']
    return food_elements

In [27]:
food_elements = get_food_elements()

---RETRIEVE---
---GENERATE---


In [28]:
food_elements

'The context mentions various food items such as cherrywood smoked bacon, apple wood bacon, fried chicken, red eye gravy, mashed potatoes, Ecuadorian chocolate tart, wide-ass salmon, Mac and Cheese, peanuts, scrambled eggs, sandwiches, and hot dogs. There are also mentions of food banks and people who are food insecure during the Covid outbreak.'

In [31]:

class Recipes(BaseModel):
    names: List = Field(description="the list of recipe names")

In [32]:
def get_recipe_names(food_elements):
    parser = PydanticOutputParser(pydantic_object=Recipes)

    prompt = PromptTemplate(
        template="Answer the user query.\n{format_instructions}\n{query}\n",
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    
    
    llm = ChatMistralAI(mistral_api_key='', model = 'mistral-tiny')
    chain = prompt | llm | parser
    recipes = chain.invoke({"query": f'''Given the following information {food_elements} 
    Generate pun-filled, humorous recipe names. 
    For example, if a video featured a giant pizza challenge, the LLM might create a recipe called "Beastly Behemoth Supreme Pizza.'''})
    return recipes
    

In [34]:
recipe_names = get_recipe_names(food_elements)

In [35]:
print(recipe_names.names)

['Cherrybacon Delight: The Wood-Smoked Wondrousness', 'Apple-wood Apple-dacious Bacon Bites', 'Fowl Play: Crispy Fried Chicken Cha-Cha', "Red-Eyed Rooster's Revenge: Red Eye Gravy Galore", 'Mashed Potato Palava: Spudtastic Surprise', 'Ecuadorian Choco-Laugh: Tarty Temptation', 'Salmon Swim: Wide-ass Wonderfish', 'Mac Attack: Cheesy Mac and Cheese Marvel', 'Peanut Pranksters: Crunchy Craving Crusaders', 'Scrambled Eggscelence: Sunny Side Up Surprise', 'Sammiches and Hot Dogs: Deli Delights', 'Hunger Helpers: Food Bank Feast', 'Covid Cravings: Recipes for the Food Insecure']


In [36]:

class Ingredients(BaseModel):
    ingredients: List[str] = Field(description="the list of ingredients")

def get_ingredients(recipe_name):
    parser = PydanticOutputParser(pydantic_object=Ingredients)
    
    prompt = PromptTemplate(
        template="Answer the user query.\n{format_instructions}\n{query}\n",
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    
    
    llm = ChatMistralAI(mistral_api_key='', model = 'mistral-tiny')
    chain = prompt | llm | parser
    ing = chain.invoke({"query": f'''Given the following  recipe name {recipe_name}
        Create ingredient lists: For each recipe name, have the LLM generate an absurd, over-the-top list of ingredients. 
        Encourage the LLM to include ridiculous quantities, unexpected flavor combinations, and ingredients that tie back to specific challenges or stunts.'''})
    return ing.ingredients



In [37]:
rps = []
for name in recipe_names.names:
    recipe = {
        "name": name
    }
    rps.append(recipe)
    

In [38]:
for r in rps:
    ingredients = get_ingredients(r['name'])
    r['ingredients'] = ingredients

In [39]:
rps[0]

{'name': 'Cherrybacon Delight: The Wood-Smoked Wondrousness',
 'ingredients': ['100 lbs of fresh, juicy cherries, pitted and halved',
  '50 lbs of crispy bacon, cooked until extra crispy',
  '2 gallons of pure maple syrup',
  '1 lb of ground cinnamon',
  '50 lbs of brown sugar',
  '100 lbs of unsalted butter',
  '50 lbs of smoked sea salt',
  '25 lbs of black pepper, freshly ground',
  '50 lbs of chili powder',
  '100 lbs of marshmallows',
  '1 ton of graham crackers, crushed',
  '50 lbs of dark chocolate, chopped',
  '10 cans of whipped cream',
  '10 bottles of hot sauce, for garnish',
  '1 can of baked beans, for added texture',
  '1 bottle of olive oil, for wood-smoking',
  '1 bag of hickory chips, for wood-smoking',
  '1 container of edible glitter, for a festive touch']}

In [40]:
class Instructions(BaseModel):
    instructions: List[str] = Field(description="the list of instructions")

In [43]:
def get_recipe_instructions(recipe):
    parser = PydanticOutputParser(pydantic_object=Instructions)
    
    prompt = PromptTemplate(
        template="Answer the user query.\n{format_instructions}\n{query}\n",
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    
    
    llm = ChatMistralAI(mistral_api_key='', model = 'mistral-small')
    chain = prompt | llm | parser
    question = f'''Given the following recipe {recipe['name']} and ingredients {recipe['ingredients']} 
        Write recipe instructions: Feed the recipe names and ingredient lists into the LLM and have it generate step-by-step instructions for each dish.
        The instructions should be a mix of practical cooking steps and humorous, exaggerated descriptions that reference Mr. Beast's antics.
        '''
    ins = chain.invoke({"query": question})
    return ins.instructions



In [44]:
for r in rps:
    instructions = get_recipe_instructions(r)
    r['instructions'] = instructions

In [46]:
for i in rps[0]['instructions']:
    print(i)
    print('-------------------------------------------------------------------------------------')

Step 1: Gather the key ingredients for Cherrybacon Delight, including a staggering 100 lbs of fresh, juicy cherries, 50 lbs of crispy bacon, 2 gallons of pure maple syrup, 1 lb of ground cinnamon, 50 lbs of brown sugar, 100 lbs of unsalted butter, 50 lbs of smoked sea salt, 25 lbs of black pepper, 50 lbs of chili powder, 100 lbs of marshmallows, 1 ton of crushed graham crackers, 50 lbs of chopped dark chocolate, 10 cans of whipped cream, 10 bottles of hot sauce, 1 can of baked beans, 1 bottle of olive oil, 1 bag of hickory chips, and 1 container of edible glitter for a festive touch. Don't forget to channel your inner Mr. Beast and think big!
-------------------------------------------------------------------------------------
Step 2: Begin by wood-smoking the cherries and bacon using the hickory chips and olive oil to infuse them with a rich, smoky flavor. As Mr. Beast would say, 'Go big or go home' – let's make this a 24-hour smoke fest to really lock in the flavors!
----------------

In [47]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from typing import List

In [48]:
class Suggestions(BaseModel):
    suggestions: List[str] = Field(description="the list of serving suggestions")

In [49]:
def get_serving_suggestions(recipe):
    parser = PydanticOutputParser(pydantic_object=Suggestions)
    
    prompt = PromptTemplate(
        template="Answer the user query.\n{format_instructions}\n{query}\n",
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    
    
    llm = ChatMistralAI(mistral_api_key='', model = 'mistral-tiny')
    chain = prompt | llm | parser
    question = f'''
        You are given the following information recipe name {recipe['name']} 
        Generate serving suggestions: Have the LLM create amusing, tongue-in-cheek serving suggestions for each recipe. 
        These could include things like "Serve on a golden platter to 10,000 of your closest friends" or "Pair with a refreshing 100-gallon soda challenge.
        '''
    ins = chain.invoke({"query": question})
    return ins.suggestions



In [50]:
for r in rps:
    suggestions = get_serving_suggestions(r)
    r['suggestions'] = suggestions

In [52]:
for r in rps:
    r['serving_suggestion'] = r['suggestions'][0]

In [51]:
rps[0]['suggestions']

['Serve Cherrybacon Delight to your royal court as a sign of your culinary excellence.',
 'Pair this Wood-Smoked Wondrousness with a refreshing 50-gallon soda challenge for a truly memorable experience.',
 'Have your guests guess the number of cherries in each bite before they take a taste.',
 'Serve Cherrybacon Delight on a golden platter to 10,000 of your closest friends.',
 'Create a cherrybacon delight flavor ice cream and serve it as a dessert.',
 'Serve Cherrybacon Delight with a side of wood-smoked laughter to enhance the flavor experience.']

In [57]:
import gradio as gr

# Define the recipe data
recipes = rps
intro = """
    <div style="text-align: center; color: #8e44ad;background-color: #f8d49e; padding: 20px; border-radius: 10px; box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.1);">
        <h1 style="color: #8e44ad; font-size: 36px; margin-bottom: 20px;">Welcome to the Beast Cookbook!</h1>
        <p style="color: #8e44ad; font-size: 18px;">Prepare yourself for a wild culinary adventure inspired by Mr. Beast's epic food challenges. 
        This cookbook is filled with larger-than-life recipes that will push your cooking skills to the limit.
        Get ready to conquer the kitchen like a true Beast!</p>
    </div>
"""

outro = """
    <div style="text-align: center; background-color: #c8d6e5; color: #8e44ad;padding: 20px; border-radius: 10px; box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.1);">
        <h2 style="color: #8e44ad; font-size: 28px; margin-bottom: 20px;">Congratulations, you've reached the end of the Beast Cookbook!</h2>
        <p style="color: #8e44ad; font-size: 18px;">We hope you've enjoyed this whimsical culinary journey inspired by Mr. Beast's over-the-top food challenges.
        Now it's time for you to take on your own kitchen adventures and create some truly unforgettable meals.
        Remember, cooking is all about having fun and pushing boundaries. So go forth and be a Beast in the kitchen!</p>
    </div>
"""

def display_recipe(recipe_index):
    if recipe_index < 0 or recipe_index >= len(recipes):
        return "Recipe not found."
    
    recipe = recipes[recipe_index]
    return f"""
        <div style="background-color: #f5f5f5; padding: 20px; border-radius: 10px; box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.1);">
            <h2 style="color: #e67e22; font-size: 32px; margin-bottom: 20px;">{recipe['name']}</h2>
            <h3 style="color: #2c3e50; font-size: 24px; margin-top: 30px;">Ingredients:</h3>
            <ul style="list-style-type: none; padding-left: 0;">
                {"".join([f'<li style="margin-bottom: 10px;color: #8e44ad;"><span style="color: #8e44ad; font-size: 18px; margin-right: 10px;">•</span>{ingredient}</li>' for ingredient in recipe['ingredients']])}
            </ul>
            <h3 style="color: #2c3e50; font-size: 24px; margin-top: 30px;">Instructions:</h3>
            <ol style="padding-left: 20px;">
                {"".join([f'<li style="margin-bottom: 10px; color: #8e44ad; font-size: 18px;">{step}</li>' for step in recipe['instructions']])}
            </ol>
            <h3 style="color: #2c3e50; font-size: 24px; margin-top: 30px;">Serving Suggestion:</h3>
            <p style="color: #d35400; font-size: 18px; font-style: italic;">{recipe['serving_suggestion']}</p>
        </div>
    """

recipe_index = 0

def cookbook(action):
    global recipe_index
    if action == "Next":
        recipe_index = (recipe_index + 1) % len(recipes)
    elif action == "Previous":
        recipe_index = (recipe_index - 1) % len(recipes)
    
    if recipe_index == 0:
        return intro + display_recipe(recipe_index)
    elif recipe_index == len(recipes) - 1:
        return display_recipe(recipe_index) + outro
    else:
        return display_recipe(recipe_index)

with gr.Blocks(theme="default") as blocks:
    # Injecting CSS for custom styling
    gr.HTML(value="""
        <style>
            body {
                font-family: Arial, sans-serif;
                background-color: #f9f9f9; /* Light grey background */
                color: #333; /* Dark grey text */
            }
            .gr-blocks-container, .gradio-container {
                max-width: 800px;
                margin: 20px auto;
                padding: 20px;
                background-color: #fff; /* White background for the container */
                border-radius: 10px;
                box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
            }
            /* Custom styles for Markdown components */
            h1, p {
                color: #31708e; /* Dark cyan text for headings and paragraphs */
            }
            h1 {
                background-color: #d9edf7; /* Light cyan background for headings */
            }
            p {
                background-color: #fefefe; /* Very light grey background for paragraphs */
            }
        </style>
    """, visible=False)
    
   
    prev_button = gr.Button("Previous")
    next_button = gr.Button("Next")
    output_html = gr.HTML()
    prev_button.click(cookbook, inputs=[prev_button], outputs=[output_html])
    next_button.click(cookbook, inputs=[next_button], outputs=[output_html])
    output_html.value=display_recipe(0)
    navigate.change(cookbook, inputs=navigate, outputs=output_html)


blocks.launch(share=True)



Running on local URL:  http://127.0.0.1:7873
Running on public URL: https://ac2aa33530bd9b22f9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
